In [50]:
import pandas as pd
import numpy as np
import os
import random

## 下面這欄要記得改喔

In [51]:
file = ["20w_25w", "25w_30w", "30w_35w", "35w_40w"]
file_range = [list(range(10200000, 10250000)), list(range(10250000, 10300000)), list(range(10300000, 10350000)),
             list(range(10350000, 10400000))]
# 讀檔路徑
read_file_path = "./訓練資料集/id"
# 存檔路徑
store_file_path = "./訓練資料集/20w_40w_top1_3"

In [38]:
data = pd.read_csv("{}/id_{}.csv".format(read_file_path, file[0]))
data.tail()
#(32975653, 53)

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
1262166,24.0,10208021.0,6,1.0,12020.337353,1.0,0.0,0.0,0.0,1.0,...,2.0,6.0,15.0,1.0,99.0,30.0,150323.167829,1.0,5.0,1.0
1262167,24.0,10208021.0,10,1.0,6786.478062,0.0,1.0,0.0,0.0,0.0,...,2.0,6.0,15.0,1.0,99.0,30.0,150323.167829,1.0,5.0,1.0
1262168,24.0,10208021.0,15,1.0,26163.698497,1.0,0.0,0.0,0.0,1.0,...,2.0,6.0,15.0,1.0,99.0,30.0,150323.167829,1.0,5.0,1.0
1262169,24.0,10208021.0,37,1.0,3335.410295,1.0,0.0,0.0,0.0,1.0,...,2.0,6.0,15.0,1.0,99.0,30.0,150323.167829,1.0,5.0,1.0
1262170,24.0,10230085.0,2,2.0,5599.605683,2.0,0.0,0.0,0.0,1.0,...,2.0,4.0,15.0,1.0,99.0,30.0,86109.761716,1.0,4.0,1.0


In [52]:
pred_tag = ['2','6','10','12','13','15','18','19','21','22','25','26','36','37','39','48']

def generate_new_data(file_name, ls):
    ### 資料處理後的dataframe
    data1 = pd.read_csv("{}/id_{}.csv".format(read_file_path, file_name))
    data_clean = pd.DataFrame({'chid':list(np.repeat(np.array(ls), 3, axis=0))})
    data_clean[list(map(lambda x:x+"_top1", list(data1.columns[2:43])))] = 0
    data_clean[list(map(lambda x:x+"_top2", list(data1.columns[2:43])))] = 0
    data_clean[list(map(lambda x:x+"_top3", list(data1.columns[2:43])))] = 0
    data_clean[list(data1.columns[43:53])] = 0
    data_clean['dt_time'] = [1,2,3]*50000
    data_clean['predtag_na'] = 3
    data_clean[['predtag_1','predtag_2','predtag_3']] = ""
    return data1, data_clean

In [53]:
import warnings
warnings.filterwarnings("ignore")
pd.set_option('mode.chained_assignment', None)
dt_list = [list(range(13,16)),list(range(16,19)),list(range(19,22))]

for order in range(len(file)):
    file_name, range_list = file[order], file_range[order]
    print(file_name)
    data1, data_clean = generate_new_data(file_name, range_list)
    
    for chid in file_range[order]: 
        if chid%1000==0: 
            print(chid) 
        for dt_num in range(len(dt_list)):
            dt_x = dt_list[dt_num]
            data1_chid = data1[data1['chid']==chid][data1['dt'].isin(dt_x)]
            data1_chid_sum = data1_chid.iloc[:,2:43].groupby(['shop_tag']).sum()
            d = data_clean[data_clean['chid']==chid]

            ### 沒有shop_tag
            if len(data1_chid_sum.index)==0:
                d[['shop_tag_top1','shop_tag_top2','shop_tag_top3']].iloc[dt_num,:] = random.sample(pred_tag, 3)

            ### 只有top1的shop_tag
            elif len(data1_chid_sum.index)==1:
                d['shop_tag_top1'].iloc[dt_num,:] = data1_chid_sum.index[0]
                d['txn_cnt_top1'].iloc[dt_num,:] = data1_chid_sum.iloc[0,0]
                d.iloc[dt_num,3:42] = (np.array(data1_chid_sum.iloc[0,1:]/data1_chid_sum.iloc[0,0]))
                
                d[['shop_tag_top2','shop_tag_top3']].iloc[dt_num,:] = random.sample(set(pred_tag)-set(list(data1_chid_sum.index)), 2)
                d.iloc[dt_num,124:134] = (np.array(data1[data1['chid']==chid].iloc[-1,43:53]))

            ### 只有top1~top2的shop_tag
            elif len(data1_chid_sum.index)==2:
                d['shop_tag_top1'].iloc[dt_num,:] = data1_chid_sum.index[0]
                d['txn_cnt_top1'].iloc[dt_num,:] = data1_chid_sum.iloc[0,0]
                d.iloc[dt_num,3:42] = (np.array(data1_chid_sum.iloc[0,1:]/data1_chid_sum.iloc[0,0]))

                d['shop_tag_top2'].iloc[dt_num,:] = data1_chid_sum.index[1]
                d['txn_cnt_top2'].iloc[dt_num,:] = data1_chid_sum.iloc[1,0]
                d.iloc[dt_num,44:83] = (np.array(data1_chid_sum.iloc[1,1:]/data1_chid_sum.iloc[1,0]))

                d['shop_tag_top3'].iloc[dt_num,:] = random.sample(set(pred_tag)-set(list(data1_chid_sum.index)), 1)
                d.iloc[dt_num,124:134] = (np.array(data1[data1['chid']==chid].iloc[-1,43:53]))

            ### top1~top3的shop_tag都有
            else:
                d['shop_tag_top1'].iloc[dt_num,:] = data1_chid_sum.index[0]
                d['txn_cnt_top1'].iloc[dt_num,:] = data1_chid_sum.iloc[0,0]
                d.iloc[dt_num,3:42] = (np.array(data1_chid_sum.iloc[0,1:]/data1_chid_sum.iloc[0,0]))

                d['shop_tag_top2'].iloc[dt_num,:] = data1_chid_sum.index[1]
                d['txn_cnt_top2'].iloc[dt_num,:] = data1_chid_sum.iloc[1,0]
                d.iloc[dt_num,44:83] = (np.array(data1_chid_sum.iloc[1,1:]/data1_chid_sum.iloc[1,0]))

                d['shop_tag_top3'].iloc[dt_num,:] = data1_chid_sum.index[2]
                d['txn_cnt_top3'].iloc[dt_num,:] = data1_chid_sum.iloc[2,0]
                d.iloc[dt_num,85:124] = (np.array(data1_chid_sum.iloc[2,1:]/data1_chid_sum.iloc[2,0]))
                d.iloc[dt_num,124:134] = (np.array(data1[data1['chid']==chid].iloc[-1,43:53]))

        #     if dt_num==2:
            dt_y = dt_x[-1]
            data1_chid_pred = data1[data1['chid']==chid][data1['dt']==(dt_y+1)]
            data1_chid_sum_pred = data1_chid_pred.iloc[:,2:43].groupby(['shop_tag']).sum()
        #         d = data_clean[data_clean['chid']==chid]


            if len(data1_chid_sum_pred.index)==1:
                d['predtag_na'].iloc[dt_num,:] = 2
                d.iloc[dt_num,136:139] = [data1_chid_sum_pred.index[0]]*3
            elif len(data1_chid_sum_pred.index)==2:
                d['predtag_na'].iloc[dt_num,:] = 1
                d.iloc[dt_num,136:139] = [data1_chid_sum_pred.index[0]]*2+[data1_chid_sum_pred.index[1]]
            elif len(data1_chid_sum_pred.index)>=3:
                d['predtag_na'].iloc[dt_num,:] = 0
                d.iloc[dt_num,136:139] = data1_chid_sum_pred.index[0:3]

        data_clean[data_clean['chid']==chid] = d
    
    # 存檔
    data_clean.to_csv("{}/{}_top1_3.csv".format(store_file_path, file_name))

20w_25w
10200000
10201000
10202000
10203000
10204000
10205000
10206000
10207000
10208000
10209000
10210000
10211000
10212000
10213000
10214000
10215000
10216000
10217000
10218000
10219000
10220000
10221000
10222000
10223000
10224000
10225000
10226000
10227000
10228000
10229000
10230000
10231000
10232000
10233000
10234000
10235000
10236000
10237000
10238000
10239000
10240000
10241000
10242000
10243000
10244000
10245000
10246000
10247000
10248000
10249000
25w_30w
10250000
10251000
10252000
10253000
10254000
10255000
10256000
10257000
10258000
10259000
10260000
10261000
10262000
10263000
10264000
10265000
10266000
10267000
10268000
10269000
10270000
10271000
10272000
10273000
10274000
10275000
10276000
10277000
10278000
10279000
10280000
10281000
10282000
10283000
10284000
10285000
10286000
10287000
10288000
10289000
10290000
10291000
10292000
10293000
10294000
10295000
10296000
10297000
10298000
10299000
30w_35w
10300000
10301000
10302000
10303000
10304000
10305000
10306000
10307000
1030

In [42]:
# data_clean[data_clean['chid']==chid] = d
data_clean.head()

,chid,shop_tag_top1,txn_cnt_top1,txn_amt_top1,domestic_offline_cnt_top1,domestic_online_cnt_top1,overseas_offline_cnt_top1,overseas_online_cnt_top1,domestic_offline_amt_pct_top1,domestic_online_amt_pct_top1,...,cuorg,slam,gender_code,age,primary_card,dt_time,predtag_na,predtag_1,predtag_2,predtag_3
0,10200000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,3,,,
1,10200000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,3,,,
2,10200000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,3,,,
3,10200001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,3,,,
4,10200001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,3,,,


In [224]:
d
# data_clean[data_clean['chid']==chid].iloc[dt_num,:]

,chid,shop_tag_top1,txn_cnt_top1,txn_amt_top1,domestic_offline_cnt_top1,domestic_online_cnt_top1,overseas_offline_cnt_top1,overseas_online_cnt_top1,domestic_offline_amt_pct_top1,domestic_online_amt_pct_top1,...,cuorg,slam,gender_code,age,primary_card,dt_time,predtag_na,predtag_1,predtag_2,predtag_3
2691,10100897,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,1,3,,,
2692,10100897,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,2,3,,,
2693,10100897,6,1,12144.807596,1.0,0.0,0.0,0.0,1.0,0.0,...,30.0,112010.611717,0.0,5.0,0.0,3,2,37,37,37


In [229]:
# data_clean.iloc[10100897*3,:]
data_clean[data_clean['chid']==10208021]

,chid,shop_tag_top1,txn_cnt_top1,txn_amt_top1,domestic_offline_cnt_top1,domestic_online_cnt_top1,overseas_offline_cnt_top1,overseas_online_cnt_top1,domestic_offline_amt_pct_top1,domestic_online_amt_pct_top1,...,cuorg,slam,gender_code,age,primary_card,dt_time,predtag_na,predtag_1,predtag_2,predtag_3
2691,10100897,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,1,3,,,
2692,10100897,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,2,3,,,
2693,10100897,6,1,12144.807596,1.0,0.0,0.0,0.0,1.0,0.0,...,30.0,112010.611717,0.0,5.0,0.0,3,2,37,37,37


In [5]:
# data_clean.to_csv('10w_15w_top1_3.csv')